In [217]:
import pandas as pd
import numpy as np
import import_ipynb
from unidecode import unidecode
import glob

import validItems

In [218]:
obras = glob.glob(r"L03_PLAN-PROG-SEMANAL-OBRAS- 30-01-2023 a 05-02-2023.xlsx")
obras

['L03_PLAN-PROG-SEMANAL-OBRAS- 30-01-2023 a 05-02-2023.xlsx']

In [219]:
def treat_list(valid_counties):
    municipalities_treated = []
    for i in range(len(valid_counties)):
        valid_counties[i] = unidecode(valid_counties[i])
        valid_counties[i] = valid_counties[i].lower()
        municipalities_treated.append(valid_counties[i])
    return municipalities_treated


def check_highway(valid_highways, worksheet_highways):
    for i in range(len(worksheet_highways)):
        if not worksheet_highways[i] in valid_highways:
            wrong_list.append(f"Rodovia errada na planilha: {obra[0:3]}| a rodovia não existe: {worksheet_highways[i]}")
            
        
def check_codes(valid_codes, worksheet_codes):
    for i in range(len(worksheet_codes)):
        if not worksheet_codes[i] in valid_codes:
            wrong_list.append(f"Cod de serviço errado na planilha: {obra[0:3]}| o codigo de serviço não existe: {worksheet_codes[i]}")


def check_counties(valid_counties, worksheet_counties):
    for i in range(len(worksheet_counties)):
        if not unidecode(worksheet_counties[i].strip()) in valid_counties:
            wrong_list.append(f"Municipio errado na planilha: {obra[0:3]}, o municipio {worksheet_counties[i].strip()}")
            

def check_coordinates(worksheet, collumn_name: str):
    comma = ","
    coordinates_list = []
    for i in range(len(worksheet)):
        coordinates_list.append(worksheet[collumn_name][i])
    comma_list = [ a for a in coordinates_list if comma in str(a)]

    if len(comma_list) != 0:
        for i in range(len(comma_list)):
            wrong_list.append(f"Coordenada errada na planilha: {obra[0:3]}| esta com , no valor: {comma_list[i]}")

def check_senses(worksheet, valid_senses):  
    for i in range(len(worksheet)):
        if not worksheet[i] in valid_senses:
            wrong_list.append(f"Sentido errado na planilha: {obra[0:3]}| o sentido não existe: {worksheet[i]}")


def check_batch(worksheet_list, valid_batch):
    for i in range(len(worksheet_list)):
        if not worksheet_list[i] in valid_batch:
            wrong_list.append(f"Lote errado na planilha: {obra[0:3]}| o lote não existe: {worksheet_list[i]}")


def check_empty(worksheet, collumn_name: str):
    empty_list = []
    for i in range(len(worksheet)):
        if pd.notna(worksheet[i]):
            print("Achei vazio")
            empty_list.append(worksheet[i])
        else:
            print("não achei")
    if empty_list != 0:
        wrong_list.append(f"na planilha: {obra[0:3]} | Há campos vazios na {collumn_name}| {len(empty_list)}")
            
def check_km(worksheet, collumn_name: str):
    dot = "."
    comma = ","
    km_list = []
    for i in range(len(worksheet)):
        km_list.append(worksheet[collumn_name][i])
    dot_list = [a for a in km_list if dot in str(a)]
    comma_list = [ a for a in km_list if comma in str(a)]

    if len(comma_list) != 0 or len(dot_list) != 0:
        if len(comma_list) > 0:
            for i in range(len(comma_list)):
                wrong_list.append(f"na planilha: {obra[0:3]} | Erro de KM, O KM {comma_list[i]}, esta com ,")
        else:
            for i in range(len(dot_list)):
                wrong_list.append(f"na planilha: {obra[0:3]} | Erro de KM, O KM {dot_list[i]}, esta com .")
        return True
    else:
        return False

In [220]:
wrong_list = []


for obra in obras:
    df = pd.read_excel(obra, sheet_name="DADOS")
    for i in range(len(df)):
        cod_planilha = []
        rodovia_planilha = []
        municipios_planilha = []
        sentidos = []
        lote = []
        f1 = []
        cod_planilha.append(df["CÓD. DO SERVIÇO"][i])
        rodovia_planilha.append(df["RODOVIA"][i])
        municipios_planilha.append(unidecode(df["MUNICÍPIO"][i].lower()))
        sentidos.append(df['SENTIDO'][i])
        lote.append(df["LOTE"][i])
        f1.append(df['FI1'])
        # check_batch(lote, validItems.lotes)
        # check_highway(validItems.rodovias, rodovia_planilha)
        # check_codes(validItems.codigo_servico, cod_planilha)
        # check_km(df, 'KM INICIAL')
        # check_coordinates(df,"LAT KM INICIAL")
        # check_coordinates(df,"LON KM INICIAL")
        # check_km(df, 'KM FINAL')
        # check_coordinates(df,"LAT KM FINAL")
        # check_coordinates(df,"LON KM FINAL")
        # check_senses(sentidos, validItems.sentido_validos)
        check_empty(f1, "FI1")
        # check_empty(df, "FI2")
        # check_empty(df, "FI3")
        # check_empty(df, "FI4")
        # check_empty(df, "FI5")
        # check_empty(df, "FI6")
        # check_empty(df, "FIAC")
        # check_empty(df, "ALT.")
        # check_empty(df, "EMPRESA RESPONSÁVEL")
        # check_empty(df, "RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA")
        # check_empty(df, "ART DO RESPONSÁVEL TÉCNICO")
        # check_empty(df, "NÍVEL DE SERVIÇO C/ INTERVENÇÃO")
        # check_counties(treat_list(validItems.municipios), municipios_planilha)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
print(f1)
for i in range(len(wrong_list)):
    print(wrong_list[i])

[0     N
1     N
2     S
3     N
4     N
5     S
6     S
7     N
8     S
9     N
10    N
11    N
12    N
13    S
14    S
15    N
16    N
17    S
18    S
19    N
20    N
21    N
22    N
23    N
24    N
25    N
26    N
27    S
28    S
29    S
30    S
31    S
32    S
33    N
34    N
35    N
36    N
37    N
38    N
39    N
40    N
41    N
42    S
43    S
44    S
45    N
46    N
47    N
48    N
49    N
50    N
Name: FI1, dtype: object]
